# Exploración del Dataset "Champs" - Análisis de Resultados en Fútbol

## Objetivos del Notebook

Este notebook guía a través del análisis completo del dataset "champs" de Kaggle, cubriendo desde la exploración inicial hasta la creación de dashboards interactivos. Está diseñado para seguir el programa de estudio detallado del curso.

### Contenido:
1. **Importar librerías y cargar datos**
2. **Exploración y consultas básicas**
3. **Estadística descriptiva y visualización**
4. **Visualizaciones comparativas**
5. **Preparación de datos para modelado**
6. **Implementación de regresión logística**
7. **Evaluación de métricas del modelo**
8. **Análisis de KPIs**
9. **Creación de dashboards interactivos**

---

## 1. Importar Librerías y Cargar el Dataset "Champs"

Comenzamos importando las librerías necesarias para el análisis y cargando el dataset desde un archivo CSV.

In [ ]:
# Importar librerías esenciales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Configurar visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# Cargar el dataset "champs" desde archivo CSV
# Nota: Asegúrate de que el archivo champs.csv esté en la carpeta recursos
try:
    df = pd.read_csv('../recursos/champs.csv')
    print("✅ Dataset cargado exitosamente")
    print(f"📊 Dimensiones: {df.shape}")
except FileNotFoundError:
    print("❌ Error: No se encontró el archivo champs.csv")
    print("📁 Asegúrate de que el archivo esté en la carpeta recursos")
    # Crear un dataset de ejemplo para demostración
    df = pd.DataFrame({
        'date': pd.date_range('2023-01-01', periods=100),
        'home_team': np.random.choice(['Team A', 'Team B', 'Team C'], 100),
        'away_team': np.random.choice(['Team A', 'Team B', 'Team C'], 100),
        'home_goals': np.random.poisson(1.5, 100),
        'away_goals': np.random.poisson(1.2, 100),
        'league': np.random.choice(['Liga 1', 'Liga 2'], 100)
    })
    print("📊 Usando dataset de ejemplo")

## 2. Exploración y Consultas Básicas del Dataset

Realizamos una exploración inicial para entender la estructura, tipos de datos y contenido del dataset.

In [ ]:
# Exploración básica del dataset
print("📈 INFORMACIÓN GENERAL DEL DATASET")
print("=" * 50)
print(f"Número de filas: {len(df)}")
print(f"Número de columnas: {len(df.columns)}")
print(f"Columnas: {list(df.columns)}")
print("\n📊 PRIMERAS 5 FILAS:")
print(df.head())

print("\n🔍 INFORMACIÓN DETALLADA:")
print(df.info())

print("\n📋 ESTADÍSTICAS DESCRIPTIVAS:")
print(df.describe())

In [ ]:
# Ejercicio: Conteo y filtrado de partidos por liga/temporada
print("🏆 ANÁLISIS POR LIGA")
print("=" * 30)

# Conteo de partidos por liga (si existe la columna)
if 'league' in df.columns:
    partidos_por_liga = df['league'].value_counts()
    print("Partidos por liga:")
    print(partidos_por_liga)
    
    # Visualización del conteo por liga
    plt.figure(figsize=(10, 6))
    partidos_por_liga.plot(kind='bar')
    plt.title('Número de Partidos por Liga')
    plt.xlabel('Liga')
    plt.ylabel('Número de Partidos')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Análisis temporal (si existe columna de fecha)
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    
    print("\n📅 ANÁLISIS TEMPORAL:")
    partidos_por_año = df['year'].value_counts().sort_index()
    print("Partidos por año:")
    print(partidos_por_año)
    
    # Visualización temporal
    plt.figure(figsize=(12, 6))
    partidos_por_año.plot(kind='line', marker='o')
    plt.title('Evolución del Número de Partidos por Año')
    plt.xlabel('Año')
    plt.ylabel('Número de Partidos')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

## 3. Estadística Descriptiva y Visualización de Goles

Calculamos estadísticas descriptivas básicas y creamos visualizaciones para entender la distribución de goles.

In [ ]:
# Estadística descriptiva de goles
print("⚽ ESTADÍSTICAS DE GOLES")
print("=" * 40)

# Verificar si existen las columnas de goles
goles_columns = [col for col in df.columns if 'goal' in col.lower()]
if not goles_columns:
    # Crear columnas de ejemplo si no existen
    df['home_goals'] = np.random.poisson(1.5, len(df))
    df['away_goals'] = np.random.poisson(1.2, len(df))
    goles_columns = ['home_goals', 'away_goals']

# Calcular estadísticas descriptivas
for col in goles_columns:
    print(f"\n📊 {col.upper()}:")
    print(f"Media: {df[col].mean():.2f}")
    print(f"Mediana: {df[col].median():.2f}")
    print(f"Moda: {df[col].mode().iloc[0] if not df[col].mode().empty else 'N/A'}")
    print(f"Desviación estándar: {df[col].std():.2f}")
    print(f"Mínimo: {df[col].min()}")
    print(f"Máximo: {df[col].max()}")

# Crear columna de goles totales
df['total_goals'] = df['home_goals'] + df['away_goals']
print(f"\n🎯 GOLES TOTALES POR PARTIDO:")
print(f"Media: {df['total_goals'].mean():.2f}")
print(f"Mediana: {df['total_goals'].median():.2f}")
print(f"Desviación estándar: {df['total_goals'].std():.2f}")